In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('training_datasets/bitcoin_prediction_dataset.csv', parse_dates=['date'])

# Sort by date
df = df.sort_values('date')
print(df.head(1))
# Normalize the features using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['bitcoin_price', 'sp500_price', 'gold_price', 'usd_index', 'oil_price', 'volatility_index', 'interest_rate']])

print(scaled_data[0:2])
# Create sequences for the LSTM model
def create_sequences(data, lookback, forecast_horizon=7):
    X, y = [], []
    for i in range(lookback, len(data) - forecast_horizon + 1):
        X.append(data[i - lookback:i])
        y.append(data[i:i + forecast_horizon, 0])  # Bitcoin price for the next 7 days
    return np.array(X), np.array(y)

lookback = 30  # Lookback window of 60 days (past 60 days as input)
X, y = create_sequences(scaled_data, lookback)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(len(X_train), len(X_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GaussianNoise, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2


model = Sequential()
model.add(GaussianNoise(0.1, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=l2(0.01))))
model.add(BatchNormalization())
model.add(LSTM(16, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7))

optimizer = Adam(learning_rate=5e-4)
model.compile(optimizer=optimizer, loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)
model_checkpoint = ModelCheckpoint('./best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

history = model.fit(X_train, y_train,
                    epochs=120, 
                    batch_size=16,
                    verbose = 2,
                    validation_data=(X_test, y_test), 
                    callbacks=[early_stopping, lr_reduction, model_checkpoint])

model.summary()

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

In [ ]:
# Predicting the next 7 days for the test set
predictions = model.predict(X_test)


def clean_model_prediction(y_pred):
    dummy_2d_array = np.zeros((7, 7))
    dummy_2d_array[:, 0] = y_pred
    rescaled_2d_array = scaler.inverse_transform(dummy_2d_array)
    return rescaled_2d_array[:, 0]

PRED_INDEX = 430
y_pred = clean_model_prediction(predictions[PRED_INDEX]) # choose index
y_real = clean_model_prediction(y_test[PRED_INDEX])

import matplotlib.pyplot as plt
plt.plot(y_pred)
plt.plot(y_real)
# Rescale predictions and true values back to original scale
# predictions_rescaled = scaler.inverse_transform(np.concatenate([predictions, np.zeros((predictions.shape[0], scaled_data.shape[1] - 7))], axis=1))[:, :7]
# y_test_rescaled = scaler.inverse_transform(np.concatenate([y_test, np.zeros((y_test.shape[0], scaled_data.shape[1] - 7))], axis=1))[:, :7]
# 
# # Show the predictions and the actual values for the next 7 days
# print("Predictions (next 7 days):", predictions_rescaled)
# print("True Values (next 7 days):", y_test_rescaled)
# 
# 
# 
# # Plot predictions vs true values for a specific test sample
# plt.figure(figsize=(12, 6))
# plt.plot(range(7), y_test_rescaled[0], label='True Bitcoin Price', color='blue')
# plt.plot(range(7), predictions_rescaled[0], label='Predicted Bitcoin Price', color='red')
# plt.title('Bitcoin Price Prediction for the Next 7 Days')
# plt.xlabel('Days')
# plt.ylabel('Bitcoin Price (USD)')
# plt.legend()
# plt.show()

In [ ]:
# save model
model.save('oracle_v0.3.keras')
# save scaler
from pickle import dump
dump(scaler, open('minmax_scaler.pkl', 'wb'))

In [ ]:
last_60_days = scaled_data[-40:].reshape((1, 60, 7))  # Reshape to fit the model input

# Predict the next 7 days of Bitcoin price
predicted_prices = model.predict(last_60_days)

# Rescale the predicted prices back to the original Bitcoin price scale
predicted_prices_rescaled = scaler.inverse_transform(np.concatenate([predicted_prices, np.zeros((predicted_prices.shape[0], scaled_data.shape[1] - 7))], axis=1))[:, :7]

# Print the predicted prices for the next 7 days
print("Predicted Bitcoin Prices for the next 7 days:", predicted_prices_rescaled)

# Plot the prediction
plt.figure(figsize=(12, 6))
plt.plot(range(7), predicted_prices_rescaled[0], label='Predicted Bitcoin Price', color='red')
plt.title('Bitcoin Price Prediction for the Next 7 Days')
plt.xlabel('Days')
plt.ylabel('Bitcoin Price (USD)')
plt.legend()
plt.show()

In [ ]:
df['bitcoin_price'].plot()

In [ ]:
a = []
for i in range(len(scaled_data)):
    a.append(scaled_data[i][0])
plt.plot(a)